# 2. Data cleaning and initial exploration

In this notebook we will be loading and performing an initial exploration of the dataset from the Protocols track for the Hercules challenge. This dataset consists of a series of 100 experimental protocols from the [Bio-Protocols journal](https://bio-protocol.org/Default.aspx).

## Setup
As always, we will begin by loading the logging system and setting up the path to import additional code from the _src_ folder.

In [1]:
%run __init__.py

INFO:root:Starting logger


In [2]:
from bokeh.io import output_notebook

output_notebook()

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Loading BokehJS ...

In [3]:
from herc_common import BokehHistogram

hist = BokehHistogram(color_fill="mediumslateblue", color_hover="slateblue")

In [4]:
def print_empty_cols(df):
    for col in df.columns:
        print(col)
        print('-' * len(col))
        res = df[df[col] == ''].index
        print(f"{len(res)} articles have no value for column {col}")
        print(res)
        print('\n')


## Parsing the data
Since all the _html_ files of each protocol have been scraped in the previous phase, we will begin by getting the path of all the files that will be parsed later on:

In [5]:
import glob

files_to_read = glob.glob(f"{PROTOCOLS_DIR}/*.html")
len(files_to_read)

100

We will also define a simple Protocol class that will be used to stored the parsed information from every HTML file, and provides some utility methods to be loaded in a pandas DataFrame:

In [6]:
from src.protocol import Protocol

Since not every protocol will have a value for every defined field (for example, some protocols may not have a background section), we will also define a decorator that will be used later on to specify which fields are optional. These fields will be an empty string if no value can be found for them:

In [7]:
from src.data_reader import parse_protocol

protocols = []
for file in files_to_read:
    protocol_id = os.path.basename(file).split('.')[0]
    with open(file, 'r', encoding='utf-8') as f:
        protocols.append(parse_protocol(f.read(), protocol_id))

## Creating a dataframe

Now that every protocol has been parsed, we can load them into a pandas DataFrame:

In [8]:
import pandas as pd

df = pd.DataFrame([protocol.to_dict() for protocol in protocols])
df.head()

,pr_id,title,abstract,materials,procedure,equipment,background,categories,authors
0,e16,Spot Assay for Yeast,This protocol can be used to compare the cell ...,Yeast cells|YES medium,Start cultures from a 2 day old plate. Use pi...,Multichannel Pipetman (Eppendorf)|OmniTray (V&...,,Microbiology|Microbial cell biology|Cell viabi...,Zongtian Tong
1,e49,Culture and Detection of Mycobacterium tubercu...,Mycobacterium tuberculosis (MTB) is the bacter...,Middlebrook 7H9 broth base powder (Sigma-Aldri...,Culture\n\t\t\n\n\t\t\t\tPreparing 7H9 liquid ...,"T75 tissue culture flasks (Sigma-Aldrich, cata...",,Microbiology|Microbial cell biology|Cell isola...,Ran Chen
2,e68,Isolation and Culture of Mouse Bone Marrow-der...,Bone marrow derived macrophages are a type of ...,L929 cells|RPMI 1640 medium (RPMI) (Life Techn...,Preparation of L-cell conditioned medium: cult...,Cell culture incubator|0.22 μm filter|27 g nee...,,Immunology|Immune cell isolation|Maintenance a...,Ran Chen
3,e99,Transwell Cell Migration Assay Using Human Bre...,Transwell migration assays have been widely us...,"Human MDA-MB-231 cell (ATCC, catalog number: ...",Carry MB-231 cells in DMEM with 10% FBS (use L...,Corning® Transwell® polycarbonate membrane ins...,,Cancer Biology|General technique|Cell biology ...,Yanling Chen
4,e100,Scratch Wound Healing Assay,The scratch wound healing assay has been widel...,"Human MDA-MB-231 cell line (ATCC, catalog numb...",Grow cells in DMEM supplemented with 10% FBS.|...,BD Falcon 24-well tissue culture plate (Fisher...,,Cancer Biology|Invasion & metastasis|Cell biol...,Yanling Chen


We can already see that none of the first 4 protocols have a value for the "background" field. we will analyse the data more in depth in the following sections.

## Cleaning and feature engineering

In [9]:
df.describe()

,pr_id,title,abstract,materials,procedure,equipment,background,categories,authors
count,100,100,100,100,100,100,100,100,100
unique,100,100,100,99,100,100,38,99,95
top,e1569,Detection of Hydrogen Peroxide by DAB Staining...,The protocol describes the procedure of total ...,,Collect about 4 ml of human venous blood sampl...,"24 well plates (Corning, Costar®, catalog numb...",,Immunology|Immune cell isolation|Lymphocyte|Ce...,Santosh K Panda|Balachandran Ravindran
freq,1,1,1,2,1,1,63,2,2


From the output above we can see that there is a unique id, title, abstract, procedure and equipment for each different protocol. Other fields have some values which are not unique. This could mean either that they have the same value or that they are empty. We will check later on whether they are empty or not.

For now, we will start by joining all the steps of each procedure into a new column which will be called '_full\_procedure\_cleaned_':

In [10]:
import re

def join_procedure_steps(procedure):
    return ' '.join(procedure.split('|'))

def clean(procedure):
    merged_procedure = join_procedure_steps(procedure)
    return re.sub('\s+', ' ', merged_procedure).strip()
    

df['full_procedure_cleaned'] = df['procedure'].apply(lambda x: clean(x))
df['full_procedure_cleaned'].loc[0][:500]

'Start cultures from a 2 day old plate. Use pipette tip to pick up strains and resuspend them in 1.5 ml YES medium or water. Vortex and transfer 1 ml to another microfuge tube. Test OD600 for an accurate reading, the OD should be between 0.1 and 0.5. Dilute the rest of the suspension to 16 OD600, around 1.6 x 106 cells per ml, 1,600 cells per μl. That is 4,800 or 5,000 cells per 3 μl. Spot 3 μl cells on each position. If using OmniTray, start with the 1st column (8 wells in each column). Do 5 fol'

We will finally add another column with the number of characters of each procedure:

In [11]:
df['num_chars_procedure'] = df['full_procedure_cleaned'].apply(lambda x: len(x))

## Initial exploration

As we have seen before, some of the values for the background, materials, categories and authors columns are not unique. We will se now whether they contain empty values or not:

In [12]:
df.head()

,pr_id,title,abstract,materials,procedure,equipment,background,categories,authors,full_procedure_cleaned,num_chars_procedure
0,e16,Spot Assay for Yeast,This protocol can be used to compare the cell ...,Yeast cells|YES medium,Start cultures from a 2 day old plate. Use pi...,Multichannel Pipetman (Eppendorf)|OmniTray (V&...,,Microbiology|Microbial cell biology|Cell viabi...,Zongtian Tong,Start cultures from a 2 day old plate. Use pip...,660
1,e49,Culture and Detection of Mycobacterium tubercu...,Mycobacterium tuberculosis (MTB) is the bacter...,Middlebrook 7H9 broth base powder (Sigma-Aldri...,Culture\n\t\t\n\n\t\t\t\tPreparing 7H9 liquid ...,"T75 tissue culture flasks (Sigma-Aldrich, cata...",,Microbiology|Microbial cell biology|Cell isola...,Ran Chen,Culture Preparing 7H9 liquid medium Day 1: Res...,1297
2,e68,Isolation and Culture of Mouse Bone Marrow-der...,Bone marrow derived macrophages are a type of ...,L929 cells|RPMI 1640 medium (RPMI) (Life Techn...,Preparation of L-cell conditioned medium: cult...,Cell culture incubator|0.22 μm filter|27 g nee...,,Immunology|Immune cell isolation|Maintenance a...,Ran Chen,Preparation of L-cell conditioned medium: cult...,1520
3,e99,Transwell Cell Migration Assay Using Human Bre...,Transwell migration assays have been widely us...,"Human MDA-MB-231 cell (ATCC, catalog number: ...",Carry MB-231 cells in DMEM with 10% FBS (use L...,Corning® Transwell® polycarbonate membrane ins...,,Cancer Biology|General technique|Cell biology ...,Yanling Chen,Carry MB-231 cells in DMEM with 10% FBS (use L...,2367
4,e100,Scratch Wound Healing Assay,The scratch wound healing assay has been widel...,"Human MDA-MB-231 cell line (ATCC, catalog numb...",Grow cells in DMEM supplemented with 10% FBS.|...,BD Falcon 24-well tissue culture plate (Fisher...,,Cancer Biology|Invasion & metastasis|Cell biol...,Yanling Chen,Grow cells in DMEM supplemented with 10% FBS. ...,1719


In [13]:
print_empty_cols(df)

pr_id
-----
0 articles have no value for column pr_id
Int64Index([], dtype='int64')


title
-----
0 articles have no value for column title
Int64Index([], dtype='int64')


abstract
--------
0 articles have no value for column abstract
Int64Index([], dtype='int64')


materials
---------
2 articles have no value for column materials
Int64Index([68, 87], dtype='int64')


procedure
---------
0 articles have no value for column procedure
Int64Index([], dtype='int64')


equipment
---------
0 articles have no value for column equipment
Int64Index([], dtype='int64')


background
----------
63 articles have no value for column background
Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 99],
           dtype='int64')


categories
----------
0 articles h

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


As we can see, for the materials and background columns this is the case (all non-unique values are empty ones). However, there are repeated values for the equipment and authors fields, which is expected.

If we are going to use the bakground column in future steps we have to notice that most of the protocols don't have a value for it.

We are going to see how the length of each procedure is distributed amongst the dataset:

In [14]:
df['num_chars_procedure'].describe()

count      100.000000
mean      5792.020000
std       6800.128724
min        157.000000
25%       2147.750000
50%       4180.000000
75%       6668.250000
max      54947.000000
Name: num_chars_procedure, dtype: float64

In [15]:
HIST_COLUMN = 'num_chars_procedure'
HIST_TITLE = "Procedure length distribution for the Bio-Protocols dataset"
HIST_XLABEL = "Procedure length (# of characters)"
HIST_YLABEL = "Number of protocols"

hist.load_plot(df, HIST_COLUMN, HIST_TITLE,
               HIST_XLABEL, HIST_YLABEL, True)

In [16]:
df.loc[0].full_procedure_cleaned

'Start cultures from a 2 day old plate. Use pipette tip to pick up strains and resuspend them in 1.5 ml YES medium or water. Vortex and transfer 1 ml to another microfuge tube. Test OD600 for an accurate reading, the OD should be between 0.1 and 0.5. Dilute the rest of the suspension to 16 OD600, around 1.6 x 106 cells per ml, 1,600 cells per μl. That is 4,800 or 5,000 cells per 3 μl. Spot 3 μl cells on each position. If using OmniTray, start with the 1st column (8 wells in each column). Do 5 fold serial dilution from the 1st to 5th column. Leave the 6th column empty. Transfer another 8 strains culture into the 7th column and do another 5 fold dilution.'

From the cells above we can see that the mean length of each procedure is about 5800 characters, and most of the protocols have procedures of 2000-7000 characters.

In [17]:
hist.save_plot(os.path.join(NOTEBOOK_2_RESULTS_DIR, '1_Protocol_procedure_length.svg'))

## Saving the dataframe

Finally, we are going to save the dataframe so we can use it later on in the following steps:

In [18]:
DF_FILE_PATH = os.path.join(NOTEBOOK_2_RESULTS_DIR, 'protocols_dataframe.pkl')

df.to_pickle(DF_FILE_PATH)